# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from urllib.request import urlretrieve

# 定義爬取博客來前100名暢銷書資訊函數

In [2]:
def books_crawler(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')

    items=soup.select('li.item')
    numbers=[e.select('strong')[0].text for e in items]
    links=[e.select('a')[0]['href'] for e in items]
    titles=[e.select('a')[1].text for e in items]
    authors=[e.select('a')[2].text for e in items]
    imgs=['https:'+e.select('img')[0]['src'] for e in items]
    #調整圖片尺寸大小(500*500)
    imgs=[e.split('&w')[0]+'&w=500&h=500' for e in imgs]
    #折扣+價格
    disprices=[e.select('li')[1].text.strip('優惠價：') for e in items]
    #將折扣+價格分開
    discounts,prices=[],[]

    for e in disprices:
        if '折' in e:
            discounts.append('{}折'.format(e.split('折')[0]))
            prices.append(e.split('折')[1])
        else:
            discounts.append(np.nan)
            prices.append(e)

    df=pd.DataFrame({
        '排名':numbers,
        '書名':titles,
        '作者':authors,
        '折扣':discounts,
        '價格':prices,
        '網址':links,
        '圖片':imgs
    })
    
    return df

# 抓取博客來暢銷書排名資訊

In [3]:
df=books_crawler('https://www.books.com.tw/web/sys_saletopb/books/02?attribute=30&loc=act_menu_th_46_002')
df.head()

,排名,書名,作者,折扣,價格,網址,圖片
0,1,原子習慣：細微改變帶來巨大成就的實證法則,詹姆斯‧克利爾,79折,261元,https://www.books.com.tw/products/0010822522?l...,https://im1.book.com.tw/image/getImage?i=https...
1,2,我用1檔ETF存自己的18%,陳重銘,79折,284元,https://www.books.com.tw/products/0010824151?l...,https://im2.book.com.tw/image/getImage?i=https...
2,3,大會計師教你從財報數字看懂經營本質,張明輝,79折,300元,https://www.books.com.tw/products/0010825332?l...,https://im1.book.com.tw/image/getImage?i=https...
3,4,一人公司：為什麼小而美是未來企業發展的趨勢,保羅．賈維斯,79折,300元,https://www.books.com.tw/products/0010825335?l...,https://im2.book.com.tw/image/getImage?i=https...
4,5,OKR：做最重要的事,約翰‧杜爾,79折,356元,https://www.books.com.tw/products/0010813170?l...,https://im1.book.com.tw/image/getImage?i=https...


# 抓取博客來暢銷書排名封面照

In [4]:
titles=df['書名'].values.tolist()
imgs=df['圖片'].values.tolist()

directory='博客來暢銷書封面'

if not os.path.isdir(directory):
    os.makedirs(directory)

#執行方便,以前10名封面照為例
for title,img in zip(titles[:10],imgs[:10]):
    urlretrieve(img,directory+'/{}.jpg'.format(title))
    print(title)

原子習慣：細微改變帶來巨大成就的實證法則
我用1檔ETF存自己的18%
大會計師教你從財報數字看懂經營本質
一人公司：為什麼小而美是未來企業發展的趨勢
OKR：做最重要的事
故事課1：3分鐘說18萬個故事，打造影響力
富爸爸，窮爸爸（20週年紀念版）
看穿假象、理智發聲，從問對問題開始：【全球長銷40年】美國大學邏輯思辨聖經
有錢人想的和你不一樣
走舊路，到不了新地方—徐重仁的經營筆記：關於創事業、做生意和學態度
